In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
%matplotlib inline
import seaborn as sns
import math
import pickle
import os, sys, gc, warnings, random, datetime

RSEED = 50

## Load Data

In [2]:
# Load training data
df_train_transac = pd.read_csv('./data/train_transaction.csv')
df_train_identity = pd.read_csv('./data/train_identity.csv')
df_train = pd.merge(df_train_transac,df_train_identity,on='TransactionID',how='left')

In [3]:
# Load test data
df_test_transac = pd.read_csv('./data/test_transaction.csv')
df_test_identity = pd.read_csv('./data/test_identity.csv')
df_test = pd.merge(df_test_transac,df_test_identity,on='TransactionID',how='left')

In [75]:
# combine train and test
df_total = df_train.append(df_test,sort=False)

In [76]:
df_total.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0.0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0.0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0.0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0.0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0.0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


# Feature Engineer

In [6]:
def clean_id31(df):
    df['id_31'] = df['id_31'].str.replace("([0-9\.])", "")
    df['id_31'][df['id_31'].str.contains('chrome', regex=False)==True] = 'chrome'
    df['id_31'][df['id_31'].str.contains('Samsung', regex=False)==True] = 'Samsung'
    df['id_31'][df['id_31'].str.contains('samsung', regex=False)==True] = 'Samsung'
    df['id_31'][df['id_31'].str.contains('firefox', regex=False)==True] = 'firefox'
    df['id_31'][df['id_31'].str.contains('safari', regex=False)==True] = 'safari'
    df['id_31'][df['id_31'].str.contains('opera', regex=False)==True] = 'opera'
    df['id_31'] = df['id_31'].str.replace(" ", "")
    df.loc[df['id_31'].str.contains('Generic/Android', na=False), 'id_31']  = 'Android'
    df.loc[df['id_31'].str.contains('androidbrowser', na=False), 'id_31']  = 'Android'
    df.loc[df['id_31'].str.contains('androidwebview', na=False), 'id_31']  = 'Android'
    df.loc[df['id_31'].str.contains('android', na=False), 'id_31']  = 'Android'
    df.loc[df['id_31'].str.contains('chromium', na=False), 'id_31']  = 'chrome'
    df.loc[df['id_31'].str.contains('google', na=False), 'id_31']  = 'chrome'
    df.loc[df['id_31'].str.contains('googlesearchapplication', na=False), 'id_31']  = 'chrome'
    df.loc[df['id_31'].str.contains('iefordesktop', na=False), 'id_31']  = 'ie'
    df.loc[df['id_31'].str.contains('iefortablet', na=False), 'id_31']  = 'ie'
    df.loc[df.id_31.isin(df.id_31.value_counts()[df.id_31.value_counts() < 20].index), 'id_31'] = "rare"
    return df

In [7]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    # if categorical_colunms are not given than treat object as categorical features
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [8]:
def make_dow_feature(df, offset=0, tname='TransactionDT'):
    """
    Creates a day of the week feature, encoded as 0-6. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    offset : float (default=0)
        offset (in days) to shift the start/end of a day.
    tname : str
        Name of the time column in df.
    """
    # found a good offset is 0.58
    days = df[tname] / (3600*24)        
    encoded_days = np.floor(days-1+offset) % 7
    return encoded_days

In [9]:
def make_hour_feature(df, tname='TransactionDT'):
    """
    Creates an hour of the day feature, encoded as 0-23. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    tname : str
        Name of the time column in df.
    """
    hours = df[tname] / (3600)        
    encoded_hours = np.floor(hours) % 24
    return encoded_hours

In [10]:
def make_pdc_amt_ratio(df):
    df_product_aveAmt = df.groupby(['ProductCD'])['TransactionAmt'].agg(['mean'])
    df_product_aveAmt.reset_index(inplace=True)
    df_ratio = pd.merge(df[['TransactionID','ProductCD',
                                             'TransactionAmt','isFraud']],
                           df_product_aveAmt,on='ProductCD',how='left')
    
    return df_ratio['TransactionAmt']/df_ratio['mean']

In [11]:
def make_card_id(df):
    '''
    cards_cols= ['card1', 'card2', 'card3', 'card5']
    for card in cards_cols: 
        if '1' in card: 
            df['card_id']= df[card].map(str)
        else : 
            df['card_id']+= ' '+df[card].map(str)
    '''
    cards_cols= ['card2','card3','card4','card5','card6','addr1',
                 'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11']
    df['card_id'] = df['card1'].astype(str)
    for card in cards_cols: 
        df['card_id']+= '_'+df[card].astype(str)
    return df['card_id']       

In [12]:
def high_missing_cols(df,threshold):
    """return features with high missing rate"""
    rm_cols = [col for col in df.columns 
               if df[col].isnull().mean() > threshold]
    return rm_cols
    

# Confirmed Features

In [77]:
# proved important feature
df_total = clean_id31(df_total)

In [447]:
# not proved 
#us_emails = ['gmail', 'net', 'edu']

In [78]:
# proved important feature
# clean Pemail
df_total['P_email'] = df_total['P_emaildomain'].str.split('.',expand=True)[0]
#df_total['P_email_suffix'] = df_total['P_emaildomain'].map(lambda x: str(x).split('.')[-1])
#df_total['P_email_suffix'] = df_total['P_email_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
df_total.drop('P_emaildomain',axis=1,inplace = True)

In [79]:
# proved important feature
# clean R_emaildomain
df_total['R_email'] = df_total['R_emaildomain'].str.split('.',expand=True)[0]
#df_total['R_email_suffix'] = df_total['R_emaildomain'].map(lambda x: str(x).split('.')[-1])
#df_total['R_email_suffix'] = df_total['R_email_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
df_total.drop('R_emaildomain',axis=1,inplace = True)

In [80]:
# proved important feature
df_total['screen_width'] = df_total['id_33'].str.split('x', expand=True)[0]
df_total['screen_height'] = df_total['id_33'].str.split('x', expand=True)[1]
df_total.drop('id_33',axis=1,inplace = True)

In [81]:
# proved important feature
df_total['pdc_amt_ratio'] = df_total['TransactionAmt']/df_total.groupby('ProductCD')['TransactionAmt'].transform('mean')

In [82]:
for col in ['card1','addr1']:
    df_total[col+'_fq_enc'] = df_total.groupby([col])['TransactionID'].transform('count')

In [83]:
# 识别持卡人
df_total['card_id'] = make_card_id(df_total)

In [84]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
df_total['DT'] = df_total['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
df_total['DT_hour'] = df_total['DT'].dt.hour
df_total['DT_day'] = df_total['DT'].dt.day
df_total['DT_month'] = df_total['DT'].dt.month
df_total['DT_year'] = df_total['DT'].dt.year

In [85]:
# 该持卡人当前一天刷卡次数
df_total['card_mv_day_fq'] = df_total.groupby(['card_id','DT_day','DT_month','DT_year'])['TransactionID'].transform('count')

# Test Features

In [39]:
# useless 'id_30',,'card2','card3','card5','P_email','R_email','addr2',
# 'dist1','dist2','D1','D2','D3','D4','D5','id_31'
#for col in ['D6','D7','D8','D9','DeviceInfo','screen_width']:
#    df_total[col+'_fq_enc'] = df_total.groupby([col])['TransactionID'].transform('count')

In [88]:
for col in ['DT_hour','DT_day','DT_month','card2','card1',
            'card3','card4','card5','card6']:#,
    df_total[col+'_Amt_ratio'] = df_total['TransactionAmt']/df_total.groupby([col])['TransactionAmt'].transform('mean')

In [584]:
#df_total['DeviceInfo'] = df_total['DeviceInfo'].fillna('unknown_device').str.lower()
#df_total['DeviceInfo'] = df_total['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
#df_total['DeviceInfo_version'] = df_total['DeviceInfo'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

In [591]:
# D9是小时记录D9中的NA信息同时保留做出来的小时变量
#df_total['D9'] = np.where(df_total['D9'].isna(),0,1)

array([0, 0, 0, ..., 0, 0, 1])

In [23]:
#df_total['id_30'] = df_total['id_30'].fillna('unknown_device').str.lower()
#df_total['id_30'] = df_total['id_30'].apply(lambda x: ''.join([i for i in x if i.isalpha()]))
#df['id_30_version'] = df['id_30'].apply(lambda x: ''.join([i for i in x if i.isnumeric()]))

In [593]:
#i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']
#df_total['M_sum'] = df_total[i_cols].sum(axis=1).astype(np.int8)
#df_total['M_na'] = df_total[i_cols].isna().sum(axis=1).astype(np.int8)

In [545]:
# 地址是不是持卡人常用的
#df_total['card_addr_fq'] = df_total.groupby(['card_id','addr1'])['TransactionID'].transform('count')

In [546]:
# 该持卡人当前一小时总计刷卡次数
#df_total['card_mv_hour_fq'] = df_total.groupby(['card_id','DT_hour','DT_day','DT_month','DT_year'])['TransactionID'].transform('count')

In [547]:
# 该持卡人当前一小时总刷卡次数比平均次数
#df_total['card_mv_hour_fq_ratio'] = df_total['card_mv_hour_fq']/df_total.groupby('card_id')['card_mv_hour_fq'].transform('mean')

In [548]:
# 该持卡人每小时消费总额
#df_total['card_hour_Amt'] = df_total.groupby(['card_id','DT_hour','DT_day','DT_month','DT_year'])['TransactionAmt'].transform('sum')

In [549]:
# 该持卡人当前小时消费额比平均小时消费额
#df_total['card_hour_Amt_ratio'] = df_total['card_hour_Amt']/df_total.groupby('card_id')['card_hour_Amt'].transform('mean')

In [551]:
# 该持卡人当前一天刷卡次数比平均次数
#df_total['card_mv_day_fq_ratio'] = df_total['card_mv_day_fq']/df_total.groupby('card_id')['card_mv_day_fq'].transform('mean')

In [552]:
# 该持卡人每日消费总额
#df_total['card_day_Amt'] = df_total.groupby(['card_id','DT_day','DT_month','DT_year'])['TransactionAmt'].transform('sum')

In [553]:
# 该持卡人当日消费总额比平均消费额
#df_total['card_day_Amt_ratio'] = df_total['card_day_Amt']/df_total.groupby('card_id')['card_day_Amt'].transform('mean')

In [453]:
#,'ProductCD','M4','remail_fraud_rate',
# 'R_email','card6' boost but less than card4
#for col in ['P_email','card4']:
#    df_total[col+'_fraud_rate'] = df_total.groupby([col])['isFraud'].transform('mean')

In [454]:
# proved important feature
#df_total['hour'] = make_hour_feature(df_total)

In [455]:
#df_total['hour_Amt'] = df_total['TransactionAmt']/df_total.groupby('hour')['TransactionAmt'].transform('mean')

In [421]:
#df_total['card_TAmt_ratio'] = df_total['TransactionAmt']/df_total.groupby('card_id')['TransactionAmt'].transform('mean')

In [422]:
#df_total['card_hour_fq'] = df_total.groupby(['card_id','hour'])['TransactionID'].transform('count')

In [423]:
#df_total['card_pdc_cnt'] = df_total.groupby(['card_id','ProductCD'])['TransactionID'].transform('count')

In [129]:
#df_total['dow'] = make_day_feature(df_total, offset=0.58)

In [101]:
#df_total['pdc_amt_std_ratio'] = df_total['TransactionAmt']/df_total.groupby('ProductCD')['TransactionAmt'].transform('std')

In [33]:
#df_total['id_30_OS'] = df_total['id_30'].str.split(' ',expand=True)[0]
#df_total['id_30_version'] = df_total['id_30'].str.split(' ',expand=True)[1]
#df_total['is_win8_vista'] = (df_total.id_30_OS == 'Windows')&((df_total.id_30_version == '8')| (df_total.id_30_version == 'Vista'))
#df_total['is_windows_otheros'] = (df_total.DeviceInfo == 'Windows')&((df_total.id_30_OS == 'Linux')| (df_total.id_30_OS == 'other'))
#df_total.drop('id_30',axis=1,inplace = True)

In [130]:
# # proved important feature
#df_total['pdc_D1_ratio'] = df_total['D1']/df_total.groupby('ProductCD')['D1'].transform('mean')

In [34]:
# proved important feature
#df_total['pdc_D2_ratio'] = df_total['D2']/df_total.groupby('ProductCD')['D2'].transform('mean')

In [32]:
#df_total['pdc_D3_ratio'] = df_total['D3']/df_total.groupby('ProductCD')['D3'].transform('mean')

In [33]:
#df_total['pdc_D4_ratio'] = df_total['D4']/df_total.groupby('ProductCD')['D4'].transform('mean')

In [35]:
# proved important feature
#df_total['pdc_D1_std_ratio'] = df_total['D1']/df_total.groupby('ProductCD')['D1'].transform('std')

In [36]:
# proved important feature
#df_total['pdc_D2_std_ratio'] = df_total['D2']/df_total.groupby('ProductCD')['D2'].transform('std')

In [36]:
# not so important
#df_total['pdc_D3_std_ratio'] = df_total['D3']/df_total.groupby('ProductCD')['D3'].transform('std')

In [37]:
#df_total['pdc_D4_std_ratio'] = df_total['D4']/df_total.groupby('ProductCD')['D4'].transform('std')

In [144]:
# proved important feature
#df_total['card_TAmt_std_ratio'] = df_total['TransactionAmt'] / df_total.groupby(['card_id'])['TransactionAmt'].transform('std')

In [145]:
#df_total['card_freq_pdc'] = df_total.groupby('card_id')['ProductCD'].transform(lambda x:x.value_counts().index[0])
#df_total['is_card_freq_pdc'] = (df_total.ProductCD == df_total.card_freq_pdc)
#df_total.drop(['card_freq_pdc'],axis=1,inplace=True)

In [147]:
#df_total['card_freq_addr1'] = df_total.groupby('card_id')['addr1'].transform(lambda x: x.value_counts(dropna=False).index[0])
#df_total['is_card_freq_addr1'] = (df_total.addr1 == df_total.card_freq_addr1)
#df_total.drop(['card_freq_addr1'],axis=1,inplace=True)

In [75]:
#df_total['card1_count'] = df_total['card1'].map(df_total['card1'].value_counts(dropna=False)).head()

In [173]:
#df_total['card_id_02_mean'] = df_total['id_02'] / df_total.groupby(['card_id'])['id_02'].transform('mean')

In [174]:
#df_total['card_id_02_std'] = df_total['id_02'] / df_total.groupby(['card_id'])['id_02'].transform('std')

In [177]:
# not so important
#df_total['card_D1_mean'] = df_total['D1'] / df_total.groupby(['card_id'])['D1'].transform('mean')

In [178]:
# not so important
#df_total['card_D2_mean'] = df_total['D2'] / df_total.groupby(['card_id'])['D2'].transform('mean')

In [179]:
#df_total['card_D3_mean'] = df_total['D3'] / df_total.groupby(['card_id'])['D3'].transform('mean')

In [180]:
#df_total['card_D4_mean'] = df_total['D4'] / df_total.groupby(['card_id'])['D4'].transform('mean')

In [181]:
# proved important feature
#df_total['card_D15_mean'] = df_total['D15'] / df_total.groupby(['card_id'])['D15'].transform('mean')

In [182]:
#df_total['card_D1_std'] = df_total['D1'] / df_total.groupby(['card_id'])['D1'].transform('std')

In [86]:
# not so important
#df_total['card_D2_std'] = df_total['D2'] / df_total.groupby(['card_id'])['D2'].transform('std')

In [87]:
#df_total['card_D3_std'] = df_total['D3'] / df_total.groupby(['card_id'])['D3'].transform('std')

In [88]:
#df_total['card_D4_std'] = df_total['D4'] / df_total.groupby(['card_id'])['D4'].transform('std')

In [ ]:
# proved important feature
#df_total['card_D15_std'] = df_total['D15'] / df_total.groupby(['card_id'])['D15'].transform('std')

In [193]:
#df_total['addr1_D15_mean'] = df_total['D15'] / df_total.groupby(['addr1'])['D15'].transform('mean')

In [194]:
# proved important feature
#df_total['addr1_D15_std'] = df_total['D15'] / df_total.groupby(['addr1'])['D15'].transform('std')

In [160]:
# proved important feature
# decimal part of the transaction amount
#df_total['TransactionAmt_decimal'] = ((df_total['TransactionAmt'] - df_total['TransactionAmt'].astype(int)) * 1000).astype(int)

In [57]:
#df_total['Device_name'] = df_total['DeviceInfo'].str.split('/', expand=True)[0]
#df_total['Device_version'] = df_total['DeviceInfo'].str.split('/', expand=True)[1]
#df_total.drop('DeviceInfo',axis=1,inplace = True)

In [58]:
#df_total.loc[df_total['Device_name'].str.contains('SM', na=False), 'Device_name']  = 'Samsung'
#df_total.loc[df_total['Device_name'].str.contains('SAMSUNG', na=False), 'Device_name']  = 'Samsung'
#df_total.loc[df_total['Device_name'].str.contains('GT-', na=False), 'Device_name']  = 'Samsung'
#df_total.loc[df_total['Device_name'].str.contains('Moto G', na=False), 'Device_name']  = 'Motorola'
#df_total.loc[df_total['Device_name'].str.contains('Moto', na=False), 'Device_name']  = 'Motorola'
#df_total.loc[df_total['Device_name'].str.contains('moto', na=False), 'Device_name']  = 'Motorola'
#df_total.loc[df_total['Device_name'].str.contains('LG-', na=False), 'Device_name']  = 'LG'
#df_total.loc[df_total['Device_name'].str.contains('rv:', na=False), 'Device_name']  = 'RV'
#df_total.loc[df_total['Device_name'].str.contains('HUAWEI', na=False), 'Device_name']  = 'Huawei'
#df_total.loc[df_total['Device_name'].str.contains('ALE-', na=False), 'Device_name']  = 'Huawei'
#df_total.loc[df_total['Device_name'].str.contains('-L', na=False), 'Device_name']  = 'Huawei'
#df_total.loc[df_total['Device_name'].str.contains('Blade', na=False), 'Device_name']  = 'ZTE'
#df_total.loc[df_total['Device_name'].str.contains('BLADE', na=False), 'Device_name']  = 'ZTE'
#df_total.loc[df_total['Device_name'].str.contains('XT', na=False), 'Device_name']  = 'Sony'
#df_total.loc[df_total.Device_name.isin(df_total.Device_name.value_counts()[df_total.Device_name.value_counts() < 200].index), 'Device_name'] = "Others"

In [95]:
# not so important
#df_total['card_freq_Device'] = df_total.groupby('card_id')['Device_name'].transform(lambda x: x.value_counts(dropna=False).index[0])
#df_total['is_card_freq_Device'] = (df_total.Device_name == df_total.card_freq_Device)

In [161]:
#df_total['is_wide'] = df_total['screen_width'] > df_total['screen_height']
#df_total['is_long'] = df_total['screen_width'] < df_total['screen_height']
#df_total['is_zero'] = (df_total['screen_width'] == 0)

In [89]:
# this feature lead to over fitting
df_total.drop('card_id',axis=1,inplace=True)
#df_total.drop('hour',axis=1,inplace=True)
df_total.drop('DT',axis=1,inplace=True)
df_total.drop('DT_hour',axis=1,inplace=True)
df_total.drop('DT_day',axis=1,inplace=True)
df_total.drop('DT_month',axis=1,inplace=True)
df_total.drop('DT_year',axis=1,inplace=True)
#df_total.drop(['card_freq_Device'],axis=1,inplace=True)

https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-corrected

# Remove Features

In [126]:
high_miss_cols = high_missing_cols(df_total,0.9)

In [127]:
one_value_cols = [col for col in df_total.columns 
                  if df_total[col].nunique() <= 1]

In [128]:
big_top_value_cols = [col for col in df_total.columns 
                      if df_total[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [129]:
cols_to_drop = list(set(high_miss_cols + one_value_cols + big_top_value_cols))

In [130]:
len(cols_to_drop)

71

In [131]:
df_total.drop(cols_to_drop, axis=1, inplace=True)

# Encoder Categorical

In [90]:
df_total_final,colname = label_encoder(df_total, categorical_columns=None)

In [91]:
df_total_final.shape

(1097231, 449)

# Save Final Features

In [102]:
#df_total_final.to_csv('./data/features476.csv', index = False)

In [92]:
with open('./data/features449.pickle', 'wb') as handle:
    pickle.dump(df_total, handle, protocol=pickle.HIGHEST_PROTOCOL)